In [88]:
from textractor import Textractor
from textractor.data.constants import Textract_API, TextractFeatures
from textractor.data.text_linearization_config import TextLinearizationConfig

In [89]:
extractor = Textractor(region_name="eu-west-1")

In [136]:
markdown_text = ""
markdown_text_config = ""

In [153]:
textract_json = extractor.start_document_analysis(
    file_source="s3://s3sagemakerbucket/textract_analysis/input_page.png",
    features=[TextractFeatures.LAYOUT, TextractFeatures.TABLES],
    save_image=False,
)

In [144]:
response_textract_async = extractor.get_result(job_id=textract_json.job_id, api=Textract_API.ANALYZE)

In [145]:
document = response_textract_async

In [126]:
print(type(document))

<class 'textractor.entities.document.Document'>


In [ ]:
#markdown analysis 1

from textractor.data.text_linearization_config import TextLinearizationConfig

config = TextLinearizationConfig(
    hide_figure_layout=False,
    title_prefix="<titles><<title>><title>",
    title_suffix="</title><</title>>",
    # hide_header_layout=True,
    section_header_prefix="<headers><<header>><header>",
    section_header_suffix="</header><</header>>",
    table_prefix="<tables><table>",
    table_suffix="</table>",
    list_layout_prefix="<<list>><list>",
    list_layout_suffix="</list><</list>>",
    # hide_footer_layout=True,
    # hide_page_num_layout=True,
)

print(document.pages[1].get_text(config=config))

In [100]:
import numpy as np
def strip_newline(cell):
    """
    A utility function to strip newline characters from a cell.
    Parameters:
    cell (str): The cell value.
    Returns:
    str: The cell value with newline characters removed.
    """
    return str(cell).strip()

def layout_table_to_excel(document, ids,csv_seperator):    
    """
    Converts an Excel table from a document to a Pandas DataFrame, 
    handling duplicated values across merged cells.

    Args:
        document: Document containing Excel table 
        ids: ID of the Excel table in the document
        csv_seperator: Separator for CSV string conversion

    Returns: 
        Pandas DataFrame representation of the Excel table
    """
    # save the table in excel format to preserve the structure of any merged cells
    buffer = io.BytesIO()    
    document.tables[ids].to_excel(buffer)
    buffer.seek(0)
    # Load workbook, get active worksheet
    wb = openpyxl.load_workbook(buffer)
    worksheet = wb.active
    # Unmerge cells, duplicate merged values to individual cells
    all_merged_cell_ranges: list[CellRange] = list(
            worksheet.merged_cells.ranges
        )
    for merged_cell_range in all_merged_cell_ranges:
        merged_cell: Cell = merged_cell_range.start_cell
        worksheet.unmerge_cells(range_string=merged_cell_range.coord)
        for row_index, col_index in merged_cell_range.cells:
            cell: Cell = worksheet.cell(row=row_index, column=col_index)
            cell.value = merged_cell.value
    # determine table header index
    df = pd.DataFrame(worksheet.values)
    df=df.map(strip_newline)
    df0=df.to_csv(sep=csv_seperator,index=False, header=None)
    row_count=len([x for x in df0.split("\n") if x])
    if row_count>1:
        if not all(value.strip() == '' for value in df0.split("\n")[0].split(csv_seperator)): 
            row_count=1
    # attach table column names
    column_row=0 if row_count==1 else 1
    df.columns = df.iloc[column_row] 
    df = df[column_row+1:]
    return df

def split_list_items_(items):
    """
    Splits the given string into a list of items, handling nested lists.

    Parameters:
    items (str): The input string containing items and possibly nested lists.

    Returns:
    list: A list containing the items extracted from the input string.
    """
    parts = re.split("(<<list>><list>|</list><</list>>)", items)  
    output = []

    inside_list = False
    list_item = ""

    for p in parts:
        if p == "<<list>><list>":
            inside_list = True    
            list_item=p
        elif p == "</list><</list>>":
            inside_list = False
            list_item += p
            output.append(list_item)
            list_item = "" 
        elif inside_list:
            list_item += p.strip()
        else:
            output.extend(p.split('\n'))
    return output

In [101]:
import io
import openpyxl
import re
import pandas as pd

"""
This script processes a document containing tables and text. It converts the tables into CSV format 
and wraps them with XML tags for easy identification. The document structure with text and tables is maintained.
"""
csv_seperator="|" #"\t"
document_holder={}
table_page={}
count=0
# Whether to handle merged cells by duplicating merged value across corresponding individual cells
unmerge_span_cells=True 
# Loop through each page in the document
for ids,page in enumerate(document.pages):
    table_count=len([word for word in page.get_text(config=config).split() if "<tables><table>" in word]) # get the number of table in the extracted document page by header we set earlier
    assert table_count==len(page.tables) # check that number of tables per page is same as *tables extracted by textract TABLE feature
    content=page.get_text(config=config).split("<tables>")
    document_holder[ids]=[]    
    for idx,item in enumerate(content):
        if "<table>" in item:           
            if unmerge_span_cells:
                df=layout_table_to_excel(document, count,csv_seperator)
            else:
                df0=  document.tables[count].to_pandas(use_columns=False).to_csv(header=False, index=None,sep=csv_seperator)
                row_count=len([x for x in df0.split("\n") if x]) #Check the number of rows in the parsed table to determine how to read the table headers. if table row count is 1 then headers is obviously at 0 else headers may or may not be at 0
                #Check if the first row in the csv is empty headers
                if row_count>1:
                    if not all(value.strip() == '' for value in df0.split("\n")[0].split(csv_seperator)): 
                        row_count=1
                df=pd.read_csv(io.StringIO(df0), sep=csv_seperator, 
                               header=0 if row_count==1 else 1, keep_default_na=False) # read table with appropiate column headers
                df.rename(columns=lambda x: '' if str(x).startswith('Unnamed:') else x, inplace=True) 
            table=df.to_csv(index=None, sep=csv_seperator)

            if ids in table_page:
                table_page[ids].append(table)
            else:
                table_page[ids]=[table]
            # Extract table data and remaining content
            pattern = re.compile(r'<table>(.*?)(</table>)', re.DOTALL) 
            data=item
            table_match = re.search(pattern, data)
            table_data = table_match.group(1) if table_match else '' 
            remaining_content = data[table_match.end():] if table_match else data            
            content[idx]=f"<<table>><table>{table}</table><</table>>" ## attach xml tags to differentiate table from other text
            count+=1
            # Check for list items in remaining content
            if "<<list>>" in remaining_content:
                output=split_list_items_(remaining_content)
                output=[x.strip() for x in output if x.strip()]
                document_holder[ids].extend([content[idx]]+output)           
            else:
                document_holder[ids].extend([content[idx]]+[x.strip() for x in remaining_content.split('\n') if x.strip()]) # split other text by new line to be independent items in the python list.
        else:   
            # Check for list items and tables in remaining content
            if "<<list>>" in item and "<table>" not in item:   
                output=split_list_items_(item)
                output=[x.strip() for x in output if x.strip()]
                document_holder[ids].extend(output)
            else:
                document_holder[ids].extend([x.strip() for x in item.split("\n") if x.strip()])

In [ ]:
text=""
for item in range(0, len(document_holder)):
    print(document_holder[item])

In [103]:
# # Flatten the nested list document_holder into a single list and Join the flattened list by "\n"
flattened_list = [item for sublist in document_holder.values() for item in sublist]
result = "\n".join( flattened_list)
header_split=result.split("<titles>")

def sub_header_content_splitta(string):   
    """
    Splits the input string by XML tags and returns a list containing the segments of text,
    excluding segments containing specific XML tags such as "<header>", "<list>", or "<table>".

    Parameters:
    string (str): The input string to be processed.

    Returns:
    list: A list containing the segments of text extracted from the input string.
    """ 
    pattern = re.compile(r'<<[^>]+>>')
    segments = re.split(pattern, string)
    result = []
    for segment in segments:
        if segment.strip():
            if "<header>" not in segment and "<list>" not in segment and  "<table>" not in segment:
                segment=[x.strip() for x in segment.split('\n') if x.strip()]
                result.extend(segment)
            else:
                result.append(segment)
    return result

In [104]:
import re
import pandas as pd
from io import StringIO

max_words = 200
chunks = {}
table_header_dict={} 
chunk_header_mapping={}
list_header_dict={}

# iterate through each title section
for title_ids, items in enumerate(header_split):
    title_chunks = []
    current_chunk = []
    num_words = 0   
    table_header_dict[title_ids]={}
    chunk_header_mapping[title_ids]={}
    list_header_dict[title_ids]={}
    chunk_counter=0
    for item_ids,item in enumerate(items.split('<headers>')): #headers
        # print("".join(current_chunk).strip())
        lines=sub_header_content_splitta(item)             
        SECTION_HEADER=None 
        TITLES=None
        num_words = 0  
        for ids_line,line in enumerate(lines): #header lines  
            
            if line.strip():
                if "<title>" in line:   
                    TITLES=re.findall(r'<title>(.*?)</title>', line)[0].strip()
                    line=TITLES 
                    if re.sub(r'<[^>]+>', '', "".join(lines)).strip()==TITLES:
                        chunk_header_mapping[title_ids][chunk_counter]=lines
                        chunk_counter+=1
                if "<header>" in line:   
                    SECTION_HEADER=re.findall(r'<header>(.*?)</header>', line)[0].strip()
                    line=SECTION_HEADER    
                    first_header_portion=True
                next_num_words = num_words + len(re.findall(r'\w+', line))  

                if  "<table>" not in line and "<list>" not in line:
                    if next_num_words > max_words and "".join(current_chunk).strip()!=SECTION_HEADER and current_chunk and "".join(current_chunk).strip()!=TITLES:
                
                        if SECTION_HEADER :
                            if first_header_portion:
                                first_header_portion=False                                            
                            else:
                                current_chunk.insert(0, SECTION_HEADER.strip())                       
                        
                        title_chunks.append(current_chunk)                  
                        chunk_header_mapping[title_ids][chunk_counter]=lines
               
                        current_chunk = []
                        num_words = 0 
                        chunk_counter+=1
             
                    current_chunk.append(line)    
                    num_words += len(re.findall(r'\w+', line))

                """
                Goal is to segment out table items and chunks intelligently.
                We chunk the table by rows and for each chunk of the table we append the table column headers
                and table headers if any. This way we preserve the table information across each chunks.
                This will help improve semantic search where all the chunks relating to a table would be in the 
                top k=n response giving the LLM mcomplet information on the table.
                """

                if "<table>" in line:
                    # Get table header which is usually line before table in document              
                    line_index=lines.index(line)
                    if line_index!=0 and "<table>" not in lines[line_index-1] and "<list>" not in lines[line_index-1]: #Check if table is first item on the page, then they wont be a header (header may be included it table) and also if table is the the last item in the list
                        header=lines[line_index-1].replace("<header>","").replace("</header>","")
                    else:
                        header=""                   
              
                    table = line.split("<table>")[-1].split("</table>")[0] # get table from demarcators              
                    df=pd.read_csv(io.StringIO(table), sep=csv_seperator, keep_default_na=False,header=None)
                    df.columns = df.iloc[0]
                    df = df[1:]
                    df.rename(columns=lambda x: '' if str(x).startswith('Unnamed:') else x, inplace=True)                    
                    table_chunks = []
                    curr_chunk = [df.columns.to_list()] #start current chunk with table column names    
                    words=len(re.findall(r'\w+', str(current_chunk)+" "+str(curr_chunk)))  
                    # Iterate through the rows in the table
                    for row in df.itertuples(index=False):
                        curr_chunk.append(row)         
                        words+=len(re.findall(r'\w+', str(row)))
                        if words > max_words:                        
                            if [x for x in table_header_dict[title_ids] if chunk_counter == x]:
                                table_header_dict[title_ids][chunk_counter].extend([header]+[table])
                            else:
                                table_header_dict[title_ids][chunk_counter]=[header]+[table]                            
                            table_chunks.append("\n".join([csv_seperator.join(str(x) for x in curr_chunk[0])] + [csv_seperator.join(str(x) for x in r) for r in curr_chunk[1:]])) #join chunk lines together to for a csv 
                            tab_chunk="\n".join([csv_seperator.join(str(x) for x in curr_chunk[0])] + [csv_seperator.join(str(x) for x in r) for r in curr_chunk[1:]]) #join chunk lines together to for a csv
                            words = len(re.findall(r'\w+', str(curr_chunk[0]))) # set word count to word length of column header names
                            if header: #If header  attach header to table                         
                                if current_chunk and current_chunk[-1].strip().lower()==header.strip().lower(): #check if header is in the chunk and remove to avoid duplicacy of header in chunk                        
                                    current_chunk.pop(-1)
                                # Append section header to table
                                if SECTION_HEADER and SECTION_HEADER.lower().strip() != header.lower().strip():
                                    if first_header_portion:
                                        first_header_portion=False
                                    else:
                                        current_chunk.insert(0, SECTION_HEADER.strip())                             
                                current_chunk.extend([header.strip()+':' if not header.strip().endswith(':') else header.strip() ]+[tab_chunk]) #enrich table header with ':'
                                title_chunks.append(current_chunk)                           
                        
                            else:
                                if SECTION_HEADER:
                                    if first_header_portion:
                                        first_header_portion=False
                                    else:
                                        current_chunk.insert(0, SECTION_HEADER.strip())                                
                                current_chunk.extend([tab_chunk])
                                title_chunks.append(current_chunk)                        
                            chunk_header_mapping[title_ids][chunk_counter]=lines
                            chunk_counter+=1
                            num_words=0
                            current_chunk=[]
                            curr_chunk = [curr_chunk[0]]
                    
                    if curr_chunk != [df.columns.to_list()] and lines.index(line) == len(lines)-1: #if table chunk still remaining and table is last item in page append as last chunk
                        table_chunks.append("\n".join([csv_seperator.join(str(x) for x in curr_chunk[0])] + [csv_seperator.join(str(x) for x in r) for r in curr_chunk[1:]]))
                        tab_chunk="\n".join([csv_seperator.join(str(x) for x in curr_chunk[0])] + [csv_seperator.join(str(x) for x in r) for r in curr_chunk[1:]])                        
                        if [x for x in table_header_dict[title_ids] if chunk_counter == x]:
                            table_header_dict[title_ids][chunk_counter].extend([header]+[table])
                        else:
                            table_header_dict[title_ids][chunk_counter]=[header]+[table]   
                        
                        if header: 
                            if current_chunk and current_chunk[-1].strip().lower()==header.strip().lower():#check if header is in the chunk and remove to avoid duplicacy of header in chunk
                                current_chunk.pop(-1) 
                            if SECTION_HEADER and SECTION_HEADER.lower().strip() != header.lower().strip():
                                if first_header_portion:
                                    first_header_portion=False
                                else:
                                    current_chunk.insert(0, SECTION_HEADER.strip())                          
                            current_chunk.extend([header.strip()+':' if not header.strip().endswith(':') else header.strip() ]+[tab_chunk])
                            title_chunks.append(current_chunk)                   
                        else:
                            if SECTION_HEADER:
                                if first_header_portion:
                                    first_header_portion=False
                                else:
                                    current_chunk.insert(0, SECTION_HEADER.strip())                            
                            current_chunk.extend([tab_chunk])
                            title_chunks.append(current_chunk)             
                        chunk_header_mapping[title_ids][chunk_counter]=lines
                        chunk_counter+=1
                        num_words=0
                        current_chunk=[]
                    elif curr_chunk != [df.columns.to_list()] and lines.index(line) != len(lines)-1: #if table is not last item in page and max word threshold is not reached, send no next loop
                        table_chunks.append("\n".join([csv_seperator.join(str(x) for x in curr_chunk[0])] + [csv_seperator.join(str(x) for x in r) for r in curr_chunk[1:]]))
                        tab_chunk="\n".join([csv_seperator.join(str(x) for x in curr_chunk[0])] + [csv_seperator.join(str(x) for x in r) for r in curr_chunk[1:]])
                        
                        if [x for x in table_header_dict[title_ids] if chunk_counter == x]:
                            table_header_dict[title_ids][chunk_counter].extend([header]+[table])
                        else:
                            table_header_dict[title_ids][chunk_counter]=[header]+[table]                         
                        if header:               
                            if current_chunk and current_chunk[-1].strip().lower()==header.strip().lower():#check if header is in the chunk and remove to avoid duplicacy of header in chunk
                                current_chunk.pop(-1) 
                            current_chunk.extend([header.strip()+':' if not header.strip().endswith(':') else header.strip() ]+[tab_chunk])
                        else:
                            current_chunk.extend([tab_chunk])                  
                        num_words=words
                     

                """
                Goal is to segment out list items and chunk intelligently.
                We chunk each list by items in the list and 
                for each list chunk we append the list header to the chunk to preserve the information of the list across chunks.
                This would boost retrieval process where question pertaining to a list will have all list chunks within
                the topK=n responses.
                """

                if "<list>" in line:
                    # Get list header which is usually line before list in document
                    line_index=lines.index(line)
                    if line_index!=0 and "<table>" not in lines[line_index-1] and "<list>" not in lines[line_index-1]: #Check if table or list is the previous item on the page, then they wont be a header
                        header=lines[line_index-1].replace("<header>","").replace("</header>","")
                    else:
                        header=""           
                    list_pattern = re.compile(r'<list>(.*?)(?:</list>|$)', re.DOTALL)   ## Grab all list contents within the list xml tags        
                    list_match = re.search(list_pattern, line)
                    list_ = list_match.group(1)
                    list_lines=list_.split("\n")                

                    curr_chunk = []  
                    words=len(re.findall(r'\w+', str(current_chunk)))  #start word count from any existing chunk
                    # Iterate through the items in the list
                    for lyst_item in list_lines:
                        curr_chunk.append(lyst_item)         
                        words+=len(re.findall(r'\w+', lyst_item)) 
                        if words >= max_words: # 
                            if [x for x in list_header_dict[title_ids] if chunk_counter == x]:
                                list_header_dict[title_ids][chunk_counter].extend([header]+[list_])
                            else:
                                list_header_dict[title_ids][chunk_counter]=[header]+[list_]  
                            words=0     
                            list_chunk="\n".join(curr_chunk)
                            if header: # attach list header                       
                                if current_chunk and current_chunk[-1].strip().lower()==header.strip().lower():#check if header is in the chunk and remove to avoid duplicacy of header in chunk                        
                                    current_chunk.pop(-1)  
                                # Append section content header to list
                                if SECTION_HEADER and SECTION_HEADER.lower().strip() != header.lower().strip():
                                    if first_header_portion:
                                        first_header_portion=False
                                    else:
                                        current_chunk.insert(0, SECTION_HEADER.strip())
                                    
                                current_chunk.extend([header.strip()+':' if not header.strip().endswith(':') else header.strip() ]+[list_chunk]) 
                                title_chunks.append(current_chunk)                          
                         
                            else:
                                if SECTION_HEADER:
                                    if first_header_portion:
                                        first_header_portion=False
                                    else:
                                        current_chunk.insert(0, SECTION_HEADER.strip())
                                    
                                current_chunk.extend([list_chunk])
                                title_chunks.append(current_chunk)                            
                            chunk_header_mapping[title_ids][chunk_counter]=lines
                            chunk_counter+=1
                            num_words=0
                            current_chunk=[]
                            curr_chunk = []
                    if curr_chunk  and lines.index(line) == len(lines)-1: #if list chunk still remaining and list is last item in page append as last chunk
                        list_chunk="\n".join(curr_chunk)
                        if [x for x in list_header_dict[title_ids] if chunk_counter == x]:
                            list_header_dict[title_ids][chunk_counter].extend([header]+[list_])
                        else:
                            list_header_dict[title_ids][chunk_counter]=[header]+[list_]  
                        if header: 
                            if current_chunk and current_chunk[-1].strip().lower()==header.strip().lower(): #check if header is in the chunk and remove to avoid duplicacy of header in chunk
                                current_chunk.pop(-1)                            
                            if SECTION_HEADER and SECTION_HEADER.lower().strip() != header.lower().strip():
                                if first_header_portion:
                                    first_header_portion=False
                                else:
                                    current_chunk.insert(0, SECTION_HEADER.strip())                   
                            current_chunk.extend([header.strip()+':' if not header.strip().endswith(':') else header.strip() ]+[list_chunk])
                            title_chunks.append(current_chunk)                        
                        else:
                            if SECTION_HEADER:
                                if first_header_portion:
                                    first_header_portion=False
                                else:
                                    current_chunk.insert(0, SECTION_HEADER.strip())                   
                            current_chunk.extend([list_chunk])
                            title_chunks.append(current_chunk)                     
                        chunk_header_mapping[title_ids][chunk_counter]=lines
                        chunk_counter+=1
                        num_words=0
                        current_chunk=[]
                    elif curr_chunk and lines.index(line) != len(lines)-1: #if list is not last item in page and max word threshold is not reached, send to next loop          
                        list_chunk="\n".join(curr_chunk)
                        if [x for x in list_header_dict[title_ids] if chunk_counter == x]:
                            list_header_dict[title_ids][chunk_counter].extend([header]+[list_])
                        else:
                            list_header_dict[title_ids][chunk_counter]=[header]+[list_]  
                        if header:               
                            if current_chunk and current_chunk[-1].strip().lower()==header.strip().lower():#check if header is in the chunk and remove to avoid duplicacy of header in chunk
                                current_chunk.pop(-1) 
                            current_chunk.extend([header.strip()+':' if not header.strip().endswith(':') else header.strip() ]+[list_chunk])
                        else:
                            current_chunk.extend([list_chunk])                  
                        num_words=words


        if current_chunk and "".join(current_chunk).strip()!=SECTION_HEADER and "".join(current_chunk).strip()!=TITLES:
    
            if SECTION_HEADER:
                if first_header_portion:
                    first_header_portion=False
                else:
                    current_chunk.insert(0, SECTION_HEADER.strip())         
            title_chunks.append(current_chunk)
            chunk_header_mapping[title_ids][chunk_counter]=lines
            current_chunk=[]
            chunk_counter+=1
    if current_chunk:
  
        title_chunks.append(current_chunk) 
        chunk_header_mapping[title_ids][chunk_counter]=lines
    chunks[title_ids] = title_chunks

In [120]:
len(chunks[0])

18

In [ ]:
# Print chunks per title section
for i, chunk in enumerate(chunks[0][0:17], start=0):
    print(f'Chunk {i}:')
    for item in chunk:
        print(item)
    print('\n')

In [146]:
markdown_text = response_textract_async.to_markdown()

In [ ]:
print(markdown_text)

In [15]:
config = TextLinearizationConfig(
    section_header_prefix="<headers><<header>><header>",
    section_header_suffix="</header><</header>>",
)
markdown_text_config = response_textract_async.to_markdown(config = config)

In [ ]:
print(markdown_text_config)

In [131]:
with open('Egypt_EG01_Credit Agricole_using_markdown.txt', 'w', encoding = "utf-8") as outfile:
    outfile.write(markdown_text)

In [ ]:
print(response_textract_async.get_text())